# The ensemble approach is to take the data, and train each of the N classifiers on a different random arrangement of 6 landmarks. 

In [4]:
# first get all X and y data from the all_points_folds
import numpy as np 
import pickle, random 
import cv2, os

X, y = [], []

for file in os.listdir("all_points_folds"): 
    with open(f"all_points_folds/{file}", 'rb') as f: 
        X_y = pickle.load(f)
        X.append(X_y[0])
        y.append(X_y[1]) 

In [5]:
import random 
def shuffle(X, y, seed = None):
    if seed == None:  
        seed = random.randrange(0, 100)
        print(f"using seed {seed}")
    np.random.seed(seed) 
    new_X = np.concatenate([X_i for X_i in X])
    new_y = np.concatenate([y_i for y_i in y])
    N = np.random.permutation(new_X.shape[0])
    new_X = new_X[N]
    new_y = new_y[N]
    new_X = new_X.reshape(5, 20, 90, 126)
    new_y = new_y.reshape(5, 20)
    return new_X, new_y
X, y = shuffle(X, y, seed = 68)

# We will now get the landmarks for each of the NUM_MODELS models. 

In [10]:
import random 
NUM_LANDMARKS = 3 
NUM_MODELS = 50 

"""model_to_arrangements = {} 
for i in range(NUM_MODELS): 
    model_to_arrangements[i] = random.sample(list(range(21)), NUM_LANDMARKS) """

model_to_arrangements = {0 : [0, 4, 8, 12, 16, 20], []}

# After getting all configuration, create a function to take in a generic dataset and get the customized one for a model. 

In [11]:
def specify_data(dataset, landmarks): 
    """
    dataset should contain a 3D matrix of (N, 90, 126). 
    """ 
    cols = []
    for landmark in landmarks:
        cols += (np.array([0, 21, 42, 63, 84, 105]) + landmark).tolist()
    
    return dataset[:, :, tuple(cols)]  

# next we'll want to get X_train, X_test, y_train, y_test all ready for creating the models 

In [12]:
VALIDATION_FOLD_INDEX = 0 # index of the fold to be used for validation
X_test, y_test = X[VALIDATION_FOLD_INDEX], y[VALIDATION_FOLD_INDEX]
X_train = np.delete(X, VALIDATION_FOLD_INDEX, axis=0).reshape(80, 90, 126) 
y_train = np.delete(y, VALIDATION_FOLD_INDEX, axis=0).reshape(80,)

In [13]:
import tensorflow as tf 
def make_model(): 
    # use this function to create all of the models 
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=False), 
        tf.keras.layers.Dropout(0.3), 
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ])

    model.compile(loss = "binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model 

In [14]:
import os

checkpoint = tf.keras.callbacks.ModelCheckpoint("best_ensemble.h5", save_best_only=True, monitor = "val_accuracy")
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", patience=10)
def train_models(): 
    # train all of the models and get them back 
    models, val_accuracies = [], []
    for _, model_configuration in model_to_arrangements.items(): 
        train_data, test_data = specify_data(X_train, model_configuration), specify_data(X_test, model_configuration)
        model = make_model() 

        try: 
            os.remove("best_ensemble.h5") 
        except Exception as e: 
            pass 

        # train this model 
        model.fit(train_data, y_train, validation_data = (test_data, y_test), epochs = 75, callbacks = [checkpoint, early_stopping]) 

        # revert to best weights, if created 
        try: 
            model.load_weights("best_ensemble.h5")
        except Exception as e:
            pass 
        
        loss, accuracy, precision, recall = model.evaluate(test_data, y_test)
        
        val_accuracies.append(accuracy) # add the accuracy 

        models.append(model) # add the model

    return models, np.array(val_accuracies)

In [15]:
models, val_accuracies = train_models() 

Epoch 1/75
3/3 [==============================] - 3s 311ms/step - loss: 0.6896 - accuracy: 0.6125 - precision: 0.6522 - recall: 0.3947 - val_loss: 0.6865 - val_accuracy: 0.5500 - val_precision: 0.8000 - val_recall: 0.3333
Epoch 2/75
3/3 [==============================] - 0s 45ms/step - loss: 0.6842 - accuracy: 0.5875 - precision: 0.6190 - recall: 0.3421 - val_loss: 0.6840 - val_accuracy: 0.6000 - val_precision: 1.0000 - val_recall: 0.3333
Epoch 3/75
3/3 [==============================] - 0s 44ms/step - loss: 0.6831 - accuracy: 0.6375 - precision: 0.6800 - recall: 0.4474 - val_loss: 0.6804 - val_accuracy: 0.7000 - val_precision: 1.0000 - val_recall: 0.5000
Epoch 4/75
3/3 [==============================] - 0s 43ms/step - loss: 0.6872 - accuracy: 0.6375 - precision: 0.6957 - recall: 0.4211 - val_loss: 0.6781 - val_accuracy: 0.6500 - val_precision: 1.0000 - val_recall: 0.4167
Epoch 5/75
3/3 [==============================] - 0s 43ms/step - loss: 0.6751 - accuracy: 0.6625 - precision: 0.720

3/3 [==============================] - 0s 29ms/step - loss: 0.6221 - accuracy: 0.7125 - precision_3: 0.6596 - recall_3: 0.8158 - val_loss: 0.5958 - val_accuracy: 0.8500 - val_precision_3: 0.9091 - val_recall_3: 0.8333
Epoch 22/75
1/1 [==============================] - 0s 19ms/step - loss: 0.5957 - accuracy: 0.8500 - precision_3: 0.9091 - recall_3: 0.8333
Epoch 1/75
3/3 [==============================] - 2s 281ms/step - loss: 0.7062 - accuracy: 0.5625 - precision_4: 0.5254 - recall_4: 0.8158 - val_loss: 0.6985 - val_accuracy: 0.8000 - val_precision_4: 0.8333 - val_recall_4: 0.8333
Epoch 2/75
3/3 [==============================] - 0s 29ms/step - loss: 0.6946 - accuracy: 0.6500 - precision_4: 0.6000 - recall_4: 0.7895 - val_loss: 0.6964 - val_accuracy: 0.7000 - val_precision_4: 0.8000 - val_recall_4: 0.6667
Epoch 3/75
3/3 [==============================] - 0s 29ms/step - loss: 0.6901 - accuracy: 0.6500 - precision_4: 0.6000 - recall_4: 0.7895 - val_loss: 0.6946 - val_accuracy: 0.6500 - va

3/3 [==============================] - 0s 37ms/step - loss: 0.6537 - accuracy: 0.6625 - precision_6: 0.6341 - recall_6: 0.6842 - val_loss: 0.6399 - val_accuracy: 0.8000 - val_precision_6: 0.8333 - val_recall_6: 0.8333
Epoch 12/75
3/3 [==============================] - 0s 36ms/step - loss: 0.6399 - accuracy: 0.7125 - precision_6: 0.6667 - recall_6: 0.7895 - val_loss: 0.6282 - val_accuracy: 0.7500 - val_precision_6: 0.7692 - val_recall_6: 0.8333
Epoch 13/75
3/3 [==============================] - 0s 36ms/step - loss: 0.6307 - accuracy: 0.7000 - precision_6: 0.6458 - recall_6: 0.8158 - val_loss: 0.6157 - val_accuracy: 0.7500 - val_precision_6: 0.7692 - val_recall_6: 0.8333
Epoch 14/75
3/3 [==============================] - 0s 36ms/step - loss: 0.6312 - accuracy: 0.6875 - precision_6: 0.6327 - recall_6: 0.8158 - val_loss: 0.6021 - val_accuracy: 0.7500 - val_precision_6: 0.7692 - val_recall_6: 0.8333
Epoch 15/75
3/3 [==============================] - 0s 35ms/step - loss: 0.6201 - accuracy: 0

Epoch 9/75
3/3 [==============================] - 0s 31ms/step - loss: 0.6571 - accuracy: 0.6875 - precision_8: 0.7097 - recall_8: 0.5789 - val_loss: 0.6464 - val_accuracy: 0.8500 - val_precision_8: 1.0000 - val_recall_8: 0.7500
Epoch 10/75
3/3 [==============================] - 0s 33ms/step - loss: 0.6494 - accuracy: 0.6750 - precision_8: 0.6429 - recall_8: 0.7105 - val_loss: 0.6361 - val_accuracy: 0.9000 - val_precision_8: 1.0000 - val_recall_8: 0.8333
Epoch 11/75
3/3 [==============================] - 0s 125ms/step - loss: 0.6454 - accuracy: 0.6875 - precision_8: 0.6444 - recall_8: 0.7632 - val_loss: 0.6245 - val_accuracy: 0.8000 - val_precision_8: 0.8333 - val_recall_8: 0.8333
Epoch 12/75
3/3 [==============================] - 0s 32ms/step - loss: 0.6311 - accuracy: 0.7000 - precision_8: 0.6522 - recall_8: 0.7895 - val_loss: 0.6117 - val_accuracy: 0.7500 - val_precision_8: 0.7692 - val_recall_8: 0.8333
Epoch 13/75
3/3 [==============================] - 0s 31ms/step - loss: 0.6298 -

3/3 [==============================] - 0s 28ms/step - loss: 0.5896 - accuracy: 0.7125 - precision_11: 0.6596 - recall_11: 0.8158 - val_loss: 0.5789 - val_accuracy: 0.8000 - val_precision_11: 0.9000 - val_recall_11: 0.7500
Epoch 23/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6022 - accuracy: 0.7000 - precision_11: 0.6458 - recall_11: 0.8158 - val_loss: 0.5733 - val_accuracy: 0.8000 - val_precision_11: 0.9000 - val_recall_11: 0.7500
Epoch 24/75
1/1 [==============================] - 0s 19ms/step - loss: 0.5683 - accuracy: 0.8000 - precision_11: 0.9000 - recall_11: 0.7500
Epoch 1/75
3/3 [==============================] - 3s 286ms/step - loss: 0.7080 - accuracy: 0.4000 - precision_12: 0.3077 - recall_12: 0.2105 - val_loss: 0.7041 - val_accuracy: 0.4500 - val_precision_12: 0.6667 - val_recall_12: 0.1667
Epoch 2/75
3/3 [==============================] - 0s 31ms/step - loss: 0.6968 - accuracy: 0.4375 - precision_12: 0.3939 - recall_12: 0.3421 - val_loss: 0.6992 - val_accu

...

Epoch 3/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6898 - accuracy: 0.4250 - precision_37: 0.3667 - recall_37: 0.2895 - val_loss: 0.6911 - val_accuracy: 0.4000 - val_precision_37: 0.0000e+00 - val_recall_37: 0.0000e+00
Epoch 4/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6829 - accuracy: 0.5625 - precision_37: 0.5652 - recall_37: 0.3421 - val_loss: 0.6868 - val_accuracy: 0.6000 - val_precision_37: 1.0000 - val_recall_37: 0.3333
Epoch 5/75
3/3 [==============================] - 0s 29ms/step - loss: 0.6824 - accuracy: 0.5750 - precision_37: 0.5909 - recall_37: 0.3421 - val_loss: 0.6826 - val_accuracy: 0.7000 - val_precision_37: 1.0000 - val_recall_37: 0.5000
Epoch 6/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6778 - accuracy: 0.5875 - precision_37: 0.5714 - recall_37: 0.5263 - val_loss: 0.6775 - val_accuracy: 0.8000 - val_precision_37: 1.0000 - val_recall_37: 0.6667
Epoch 7/75
3/3 [==============================] - 0s 37ms/st

Epoch 19/75
3/3 [==============================] - 0s 28ms/step - loss: 0.6323 - accuracy: 0.6750 - precision_40: 0.6200 - recall_40: 0.8158 - val_loss: 0.5925 - val_accuracy: 0.7500 - val_precision_40: 0.7692 - val_recall_40: 0.8333
Epoch 20/75
3/3 [==============================] - 0s 31ms/step - loss: 0.6164 - accuracy: 0.6875 - precision_40: 0.6327 - recall_40: 0.8158 - val_loss: 0.5935 - val_accuracy: 0.7500 - val_precision_40: 0.7692 - val_recall_40: 0.8333
Epoch 21/75
1/1 [==============================] - 0s 21ms/step - loss: 0.5914 - accuracy: 0.7500 - precision_40: 0.7692 - recall_40: 0.8333
Epoch 1/75
3/3 [==============================] - 3s 283ms/step - loss: 0.6950 - accuracy: 0.4625 - precision_41: 0.3684 - recall_41: 0.1842 - val_loss: 0.6945 - val_accuracy: 0.4500 - val_precision_41: 0.6667 - val_recall_41: 0.1667
Epoch 2/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6913 - accuracy: 0.5375 - precision_41: 0.5263 - recall_41: 0.2632 - val_loss: 0.690

Epoch 12/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6163 - accuracy: 0.6625 - precision_42: 0.5965 - recall_42: 0.8947 - val_loss: 0.5971 - val_accuracy: 0.7000 - val_precision_42: 0.6875 - val_recall_42: 0.9167
Epoch 13/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6246 - accuracy: 0.6375 - precision_42: 0.5763 - recall_42: 0.8947 - val_loss: 0.5828 - val_accuracy: 0.7000 - val_precision_42: 0.6875 - val_recall_42: 0.9167
Epoch 14/75
3/3 [==============================] - 0s 28ms/step - loss: 0.6267 - accuracy: 0.6250 - precision_42: 0.5667 - recall_42: 0.8947 - val_loss: 0.5789 - val_accuracy: 0.7000 - val_precision_42: 0.6875 - val_recall_42: 0.9167
Epoch 15/75
1/1 [==============================] - 0s 18ms/step - loss: 0.5790 - accuracy: 0.7000 - precision_42: 0.7143 - recall_42: 0.8333
Epoch 1/75
3/3 [==============================] - 2s 283ms/step - loss: 0.6913 - accuracy: 0.4750 - precision_43: 0.4565 - recall_43: 0.5526 - val_loss: 0.69

Epoch 8/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6710 - accuracy: 0.6875 - precision_44: 0.7600 - recall_44: 0.5000 - val_loss: 0.6691 - val_accuracy: 0.7000 - val_precision_44: 1.0000 - val_recall_44: 0.5000
Epoch 9/75
3/3 [==============================] - 0s 27ms/step - loss: 0.6661 - accuracy: 0.6625 - precision_44: 0.7391 - recall_44: 0.4474 - val_loss: 0.6655 - val_accuracy: 0.7000 - val_precision_44: 1.0000 - val_recall_44: 0.5000
Epoch 10/75
3/3 [==============================] - 0s 28ms/step - loss: 0.6670 - accuracy: 0.7000 - precision_44: 0.7692 - recall_44: 0.5263 - val_loss: 0.6608 - val_accuracy: 0.7000 - val_precision_44: 1.0000 - val_recall_44: 0.5000
Epoch 11/75
3/3 [==============================] - 0s 28ms/step - loss: 0.6576 - accuracy: 0.7125 - precision_44: 0.7778 - recall_44: 0.5526 - val_loss: 0.6544 - val_accuracy: 0.8500 - val_precision_44: 1.0000 - val_recall_44: 0.7500
Epoch 12/75
3/3 [==============================] - 0s 28ms/step - 

Epoch 17/75
3/3 [==============================] - 0s 33ms/step - loss: 0.6184 - accuracy: 0.7125 - precision_47: 0.6531 - recall_47: 0.8421 - val_loss: 0.5813 - val_accuracy: 0.7000 - val_precision_47: 0.7500 - val_recall_47: 0.7500
Epoch 18/75
3/3 [==============================] - 0s 31ms/step - loss: 0.6051 - accuracy: 0.7250 - precision_47: 0.6739 - recall_47: 0.8158 - val_loss: 0.5868 - val_accuracy: 0.7000 - val_precision_47: 0.8000 - val_recall_47: 0.6667
Epoch 19/75
3/3 [==============================] - 0s 33ms/step - loss: 0.5993 - accuracy: 0.7375 - precision_47: 0.6889 - recall_47: 0.8158 - val_loss: 0.5879 - val_accuracy: 0.7000 - val_precision_47: 0.8000 - val_recall_47: 0.6667
Epoch 20/75
3/3 [==============================] - 0s 39ms/step - loss: 0.5964 - accuracy: 0.7375 - precision_47: 0.6977 - recall_47: 0.7895 - val_loss: 0.5846 - val_accuracy: 0.7000 - val_precision_47: 0.8000 - val_recall_47: 0.6667
Epoch 21/75
1/1 [==============================] - 0s 19ms/step 

# We will then need a function to test the model's accuracy on the test set. 
# We will also want to see whether this approach does well on our six videos. 

In [16]:
def aggregate_validation_accuracy(models, X_test, y_test):
    preds = np.zeros((X_test.shape[0], 1))
    for model, landmarks in zip(models, model_to_arrangements.values()):
        test_data = specify_data(X_test, landmarks)
        preds += model.predict(test_data)
    preds = preds / len(models)
    preds = np.round_(preds).flatten()
    return (preds == y_test).mean() 

In [17]:
aggregate_validation_accuracy(models, X_test, y_test)

0.75

In [18]:
import mediapipe as mp 
from tqdm import tqdm 
def hand_locations_general(frame, min_detection_confidence = 0.5, min_tracking_confidence = 0.5): 
    """give all landmarks"""

    hands = mp.solutions.hands.Hands(min_detection_confidence=min_detection_confidence, min_tracking_confidence=min_tracking_confidence) # MAKE SURE THIS IS ALL GOOD 
    results = hands.process(frame.astype('uint8'))
    X_locations = [0] * 42
    Y_locations = [0] * 42
    Z_locations = [0] * 42

    if results.multi_hand_landmarks:
        x = y = z = 0 
        for hand, hand_landmark in enumerate(results.multi_hand_landmarks):
            for i in range(0, 21):
                landmark = hand_landmark.landmark[i]
                X_locations[x] = landmark.x
                Y_locations[y] = landmark.y 
                Z_locations[z] = landmark.z
                x += 1; y += 1; z +=1; 
            
    hands.close()
    return np.concatenate([X_locations, Y_locations, Z_locations]) 


# create a function to pad your videos 
def pad(locations, maxlen = 90, padding = "post", truncating = "post"): 
    new_locations = locations.tolist() 
    empty_row = np.zeros((1, 126))
    for i, video in tqdm(enumerate(new_locations)): 
        if len(video) < maxlen:  
            for new_row in range(maxlen - len(video)): 
                if padding == "post": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([new_locations[i], empty_row])
                if padding == "pre": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([empty_row, new_locations[i]])

        if len(video) > maxlen: 
            if truncating == "post": 
                new_locations[i] = new_locations[i][:maxlen]
            elif truncating == "pre": 
                new_locations[i] = new_locations[i][len(video) - maxlen : ]
    return np.array(new_locations)

In [19]:
# function to ensemble predict on frames. 
def ensemble_predict(models, X_test):
    preds = np.zeros((X_test.shape[0], 1))
    for model, landmarks in tqdm(zip(models, model_to_arrangements.values())):
        test_data = specify_data(X_test, landmarks)
        preds += model.predict(test_data)
    preds = preds / len(models)
    return preds.flatten()

In [20]:
# create a function to then predict on videos 
def predict_on_video(models, path): 
    LOCATIONS = []
    cap = cv2.VideoCapture(path)
    while cap.isOpened():
        _, frame = cap.read()
        if not _: break 

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        LOCATIONS.append(hand_locations_general(frame))
        print("read frame") 
        
    print("read all frames")
    LOCATIONS = np.array([LOCATIONS])
    LOCATIONS = pad(LOCATIONS)
    return ensemble_predict(models, LOCATIONS)

In [ ]:
LOCATIONS = []
cap = cv2.VideoCapture("control_1.mov")
while cap.isOpened():
    _, frame = cap.read()
    if not _: break 

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    LOCATIONS.append(hand_locations_general(frame))
    print("read frame") 